In [ ]:
import os
import pathlib  # for filepath path tooling
import lzma  # to decompress the iCOM file
import time
import asyncio

import numpy as np  # for array tooling
import pandas as pd
import matplotlib.pyplot as plt  # for plotting

# import ipyvuetify
import traitlets
import IPython.display as display
import ipywidgets

In [ ]:
# Makes it so that any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys

In [ ]:
SITE_DIRECTORIES = {
    'rccc': {
        'monaco': pathlib.Path(r'\\monacoda\FocalData\RCCC\1~Clinical'),
        'escan': pathlib.Path(r'\\pdc\Shared\Scanned Documents\RT\PhysChecks\Logfile PDFs')
    },
    'nbcc': {
        'monaco': pathlib.Path(r'\\tunnel-nbcc-monaco\FOCALDATA\NBCCC\1~Clinical'),
        'escan': pathlib.Path(r'\\tunnel-nbcc-pdc\Shared\SCAN\ESCAN\Phys\Logfile PDFs')
    },
    'sash': {
        'monaco': pathlib.Path(r'\\tunnel-sash-monaco\Users\Public\Documents\CMS\FocalData\SASH\1~Clinical'),
        'escan': pathlib.Path(r'\\tunnel-sash-physics-server\SASH-Mosaiq-eScan\Logfile PDFs')
    }
}

In [ ]:
icom_directory = pathlib.Path(r'\\rccc-physicssvr\iComLogFiles\patients')
output_directory = pathlib.Path(r'\\pdc\PExIT\Physics\Patient Specific Logfile Fluence')

In [ ]:
# monaco_directory = pathlib.Path(r'\\monacoda\FocalData\RCCC\1~Clinical')
# pdf_directory = pathlib.Path(r'P:\Scanned Documents\RT\PhysChecks\Logfile PDFs')

In [ ]:
GRID = pymedphys.mudensity.grid()
COORDS = (GRID["jaw"], GRID["mlc"])

GAMMA_OPTIONS = {
    'dose_percent_threshold': 2,  # Not actually comparing dose though
    'distance_mm_threshold': 0.5,
    'local_gamma': True,
    'quiet': True,
    'max_gamma': 5,
}

In [ ]:
class Data(traitlets.HasTraits):
    monaco_site = traitlets.Unicode()
    escan_site = traitlets.Unicode()
    
    patient_id = traitlets.Unicode()
    delivery_timestamp = traitlets.List(traitlets.Unicode())
    plan_names = traitlets.List(traitlets.Unicode())
    
data = Data()


output = ipywidgets.Output()
def clear_output(_):
    with output:
        display.clear_output()
        
data.observe(clear_output)

In [ ]:
# data.observe?

In [ ]:
def update_file_paths(change):
    patient_id = data.patient_id
    monaco_site = data.monaco_site
    
    monaco_directory = SITE_DIRECTORIES[monaco_site]['monaco']
    
    
    all_tel_paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/*tel.1'))
    all_tel_paths = sorted(all_tel_paths, key=os.path.getmtime)

    plan_names_to_choose_from = [
        f'{path.parent.name}/{path.name}' for path in all_tel_paths
    ]
    
    icom_deliveries = list(icom_directory.glob(f'{patient_id}_*/*.xz'))
    icom_deliveries = sorted(icom_deliveries, key=os.path.getmtime)
    
    icom_files_to_choose_from = [
        path.stem for path in icom_deliveries
    ]
    
    timestamps = list(pd.to_datetime(
        icom_files_to_choose_from, format='%Y%m%d_%H%M%S').astype(str))
    
    data.delivery_timestamp = timestamps
    data.plan_names = plan_names_to_choose_from
    
    
data.observe(update_file_paths, names=['patient_id'])

In [ ]:
monaco_select = ipywidgets.SelectMultiple(
    options=data.plan_names,
    description='Monaco',
    disabled=False,
    rows=len(data.plan_names)
)

def handle_monaco_select_change(change):
    monaco_select.options = data.plan_names
    monaco_select.rows = len(data.plan_names)
    

data.observe(handle_monaco_select_change, names=['plan_names'])
monaco_select.observe(clear_output)

In [ ]:
# ipywidgets.SelectMultiple?

In [ ]:
icom_select = ipywidgets.SelectMultiple(
    options=data.delivery_timestamp,
    description='Delivery',
    disabled=False,
    rows=len(data.delivery_timestamp)
)

def handle_icom_select_change(change):
    icom_select.options = data.delivery_timestamp
    icom_select.rows = len(data.delivery_timestamp)

data.observe(handle_icom_select_change, names=['delivery_timestamp'])
icom_select.observe(clear_output)

In [ ]:
patient_id_text = ipywidgets.Text(
    description="Patient ID",
    disabled=True
)

def handle_patient_id_change(change):
    data.patient_id = change.new.zfill(6)
    
patient_id_text.observe(handle_patient_id_change, names=['value'])

In [ ]:
monaco_site_select = ipywidgets.Select(
    options=SITE_DIRECTORIES.keys(),
    value=None,
    rows=len(SITE_DIRECTORIES.keys()),
    description='Site',
    disabled=False
)

def handle_monaco_site_change(change):
    if change.new:
        patient_id_text.disabled = False
    
    patient_id_text.value = ''
    data.patient_id = ''
    
    data.monaco_site = monaco_site_select.value
    
monaco_site_select.observe(handle_monaco_site_change, names=['value'])

In [ ]:
escan_site_select = ipywidgets.Select(
    options=SITE_DIRECTORIES.keys(),
    value=None,
    rows=len(SITE_DIRECTORIES.keys()),
    description='Site',
    disabled=False
)

def handle_escan_site_change(change):    
    data.escan_site = escan_site_select.value
    
escan_site_select.observe(handle_escan_site_change, names=['value'])

In [ ]:
def to_tuple(array):
    return tuple(map(tuple, array))



In [ ]:
def plot_gamma_hist(gamma, percent, dist):
    valid_gamma = gamma[~np.isnan(gamma)]

    plt.hist(valid_gamma, 50, density=True)
    pass_ratio = np.sum(valid_gamma <= 1) / len(valid_gamma)

    plt.title(
        "Local Gamma ({0}%/{1}mm) | Percent Pass: {2:.2f} % | Mean Gamma: {3:.2f} | Max Gamma: {4:.2f}".format(
            percent, dist, pass_ratio * 100, np.mean(valid_gamma), np.max(valid_gamma)
        )
    )

In [ ]:
def plot_and_save_results(
    mudensity_tel,
    mudensity_icom,
    gamma,
    png_filepath,
    pdf_filepath,
    header_text="",
    footer_text="",
):
    diff = mudensity_icom - mudensity_tel
    largest_item = np.max(np.abs(diff))

    widths = [1, 1]
    heights = [0.3, 1, 1, 1, 0.1]
    gs_kw = dict(width_ratios=widths, height_ratios=heights)

    fig, axs = plt.subplots(5, 2, figsize=(10, 16), gridspec_kw=gs_kw)
    gs = axs[0, 0].get_gridspec()

    for ax in axs[0, 0:]:
        ax.remove()

    for ax in axs[1, 0:]:
        ax.remove()

    for ax in axs[4, 0:]:
        ax.remove()

    axheader = fig.add_subplot(gs[0, :])
    axhist = fig.add_subplot(gs[1, :])
    axfooter = fig.add_subplot(gs[4, :])

    axheader.axis("off")
    axfooter.axis("off")

    axheader.text(0, 0, header_text, ha="left", wrap=True, fontsize=30)
    axfooter.text(0, 1, footer_text, ha="left", va="top", wrap=True, fontsize=6)

    plt.sca(axs[2, 0])
    pymedphys.mudensity.display(GRID, mudensity_tel)
    axs[2, 0].set_title("Monaco Plan MU Density")

    plt.sca(axs[2, 1])
    pymedphys.mudensity.display(GRID, mudensity_icom)
    axs[2, 1].set_title("Recorded iCOM MU Density")

    plt.sca(axs[3, 0])
    pymedphys.mudensity.display(
        GRID, diff, cmap="seismic", vmin=-largest_item, vmax=largest_item
    )
    plt.title("iCOM - Monaco")

    plt.sca(axs[3, 1])
    pymedphys.mudensity.display(GRID, gamma, cmap="coolwarm", vmin=0, vmax=2)
    plt.title(
        "Local Gamma | "
        f"{GAMMA_OPTIONS['dose_percent_threshold']}%/"
        f"{GAMMA_OPTIONS['distance_mm_threshold']}mm")

    plt.sca(axhist)
    plot_gamma_hist(
        gamma, 
        GAMMA_OPTIONS['dose_percent_threshold'], 
        GAMMA_OPTIONS['distance_mm_threshold'])

    return fig

In [ ]:
def display_content(header, result):
    with output:
        display.display(
            display.Markdown(header)
        )
        display.display(result)
        
def print_markdown(markdown):
    with output:
        display.display(
            display.Markdown(markdown)
        )
        

def run_calculation():
    patient_id = data.patient_id
    
    monaco_plans = monaco_select.value
    icom_deliveries = icom_select.value
    
    monaco_site = data.monaco_site
    escan_site = data.escan_site
    
    monaco_directory = SITE_DIRECTORIES[monaco_site]['monaco']
    pdf_directory = SITE_DIRECTORIES[escan_site]['escan']
    
    print_markdown('## Output')
    
    tel_paths = []
    
    for plan in monaco_plans:
        current_plans = list(monaco_directory.glob(f'*~{patient_id}/plan/{plan}'))
        assert len(current_plans) == 1
        tel_paths += current_plans
    
    display_content('### Monaco plan paths', tel_paths)
        
    icom_paths = []

    for icom_delivery in icom_deliveries:
        icom_filename = icom_delivery.replace(' ', '_').replace('-', '').replace(':', '')
        icom_paths += list(icom_directory.glob(f'{patient_id}_*/{icom_filename}.xz'))

    
    display_content('### iCOM log file paths', icom_paths)
        
    icom_streams = []

    for icom_path in icom_paths:
        with lzma.open(icom_path, 'r') as f:
            icom_streams += [f.read()]
            
            
    deliveries_icom = []

    for icom_stream in icom_streams:
        deliveries_icom += [pymedphys.Delivery.from_icom(icom_stream)]

        
    deliveries_tel = []

    for tel_path in tel_paths:
        deliveries_tel += [pymedphys.Delivery.from_monaco(tel_path)]
        
    
    print_markdown('### Beginning calculation')
    print_markdown('Calculating Monaco MU Density...')
    mudensity_tel = deliveries_tel[0].mudensity()

    for delivery_tel in deliveries_tel[1::]:
        mudensity_tel = mudensity_tel + delivery_tel.mudensity()
        
    print_markdown('Calculating iCOM MU Density...')
    mudensity_icom = np.zeros_like(mudensity_tel)

    for path, delivery_icom in zip(icom_paths, deliveries_icom):
        mudensity_icom = mudensity_icom + delivery_icom.mudensity()
        
    print_markdown('Calculating Gamma...')
    gamma = pymedphys.gamma(
        COORDS,
        to_tuple(mudensity_tel),
        COORDS,
        to_tuple(mudensity_icom),
        **GAMMA_OPTIONS
    )
    
    print_markdown('Creating figure...')
    results_dir = output_directory.joinpath(
        patient_id, tel_path.parent.name, icom_path.stem)
    results_dir.mkdir(exist_ok=True, parents=True)

    header_text = (
        f"Patient ID: {data.patient_id}\n"
        f"Plan Name: {tel_path.parent.name}\n"
    )

    icom_path_strings = '\n    '.join([str(icom_path) for icom_path in icom_paths])
    tel_path_strings = '\n    '.join([str(tel_path) for tel_path in tel_paths])

    footer_text = (
        f"tel.1 file path(s): {tel_path_strings}\n"
        f"icom file path(s): {icom_path_strings}\n"
        f"results path: {str(results_dir)}"
    )

    png_filepath = str(results_dir.joinpath("result.png").resolve())
    pdf_filepath = str(pdf_directory.joinpath(
        f"{patient_id}-{monaco_plans[0].replace('/','-')}.pdf").resolve())

    fig = plot_and_save_results(
        mudensity_tel, mudensity_icom, 
        gamma, png_filepath, pdf_filepath, 
        header_text=header_text, footer_text=footer_text
    )

    fig.tight_layout()
    
    print_markdown('Saving figure...')
    plt.savefig(png_filepath, dpi=300)
    
    print_markdown('## Results')
    
    with output:
        plt.show()
    
    !magick convert "{png_filepath}" "{pdf_filepath}"

In [ ]:
run_calc_button = ipywidgets.Button(
    description=' Run Calculation',
    button_style='success',
    icon='play'
)

def handle_run_calc_button_press(_):
    with output:
        run_calculation()
    
run_calc_button.on_click(handle_run_calc_button_press)

# Monaco to iCOM comparison tool

Here is a tool to compare Monaco plans to the iCOM delivery log.

## Monaco Site

Choose the site to search for patient data

In [ ]:
monaco_site_select

## eSCAN Site

Chose the site to save the eSCAN PDF

In [ ]:
escan_site_select

## Patient ID

Provide the Patient ID for the plan you wish to check

In [ ]:
patient_id_text

## Monaco Plan

Select which Monaco plan to compare, multiple plans can be selected by `Ctrl + Click`.

In [ ]:
monaco_select

## iCOM Plan

Select the time(s) for a delivery that would match a single fraction of the plan selected above. As before, `Ctrl + Click` can be used to select multiple logs.

In [ ]:
icom_select

In [ ]:
run_calc_button

In [ ]:
output